# Escenarios experimentales Internetworking
Presentado por: Jonathan Toapanta
Fecha: 23/02/2023


In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Gives us Graphics
url = 'https://raw.githubusercontent.com/beespinosa1/Inter/main/Escenario1.csv'
df = pd.read_csv(url)
df= pd.read_csv(url,low_memory=False)
dataframe= pd.read_csv(url,low_memory=False)


In [70]:
# Normalise the data
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df


In [71]:
dataframe.shape
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print(list(dataframe))


        Source Port   Destination Port   Protocol   Flow Duration  \
count   3000.000000        3000.000000     3000.0    3.000000e+03   
mean   47711.509667         473.548333        6.0    7.172773e+06   
std     8791.730134         704.797817        0.0    3.617904e+06   
min    33071.000000         110.000000        6.0    3.300000e+01   
25%    36922.000000         443.000000        6.0    4.802248e+06   
50%    51024.000000         443.000000        6.0    9.343494e+06   
75%    54453.000000         443.000000        6.0    9.949760e+06   
max    60740.000000        9010.000000        6.0    1.000000e+07   

        Flow Bytes/s   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
count   3.000000e+03      3000.000000    3.000000e+03   3.000000e+03   
mean    4.154924e+05      1194.847820    2.036651e+05   4.830987e+05   
std     1.855317e+06      6043.023449    4.521030e+05   8.843235e+05   
min     0.000000e+00         0.400114    3.300000e+01   0.000000e+00   
25%     1.931392e+

In [72]:
keys = dataframe.keys()

data_to_process = dataframe[keys[4:len(keys)-1]].copy()

x_normalised=dfNormalize(data_to_process)
print(x_normalised.describe())


        Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count     3000.0     3000.000000    3000.000000      3000.000000   
mean         0.0        0.717276       0.015330         0.019709   
std          0.0        0.361792       0.068455         0.099711   
min          0.0        0.000000       0.000000         0.000000   
25%          0.0        0.480223       0.000071         0.000085   
50%          0.0        0.934349       0.000879         0.000653   
75%          0.0        0.994976       0.005350         0.003141   
max          0.0        1.000000       1.000000         1.000000   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count     3000.000000    3000.000000    3000.000000    3000.000000   
mean         0.061108       0.083702       0.223747       0.013002   
std          0.135671       0.153219       0.250063       0.031133   
min          0.000000       0.000000       0.000000       0.000000   
25%          0.001566       0.004839 

In [73]:
# get the train and test data
y = dataframe['label']
label_map = {'AUDIO': 0, 'BROWSING': 1, 
             'CHAT': 2, 'FILE-TRANSFER' : 3,
             'MAIL' : 4, 'P2P' : 5,
             'VIDEO' : 6, 'VOIP' : 7}
change_labels = lambda x: label_map[x] if x in label_map else -1
y_normalised = dataframe['label'].apply(change_labels)

In [74]:
#Import required libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [78]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(1, 3), activation='relu', input_shape=(1, x_normalised.shape[1], 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Conv2D(64, kernel_size=(1, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 2), padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

In [79]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
#Convert labels to one-hot encoding
from keras.utils import to_categorical
y_onehot = to_categorical(y_normalised, num_classes=8)

In [81]:
#Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_normalised, y_onehot, test_size=0.3, random_state=42)

In [83]:
import numpy as np

X_train = np.array(X_train)
history = model.fit(X_train.reshape(X_train.shape[0],1,x_normalised.shape[1],1), y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)


Epoch 1/50
53/53 [==============================] - 3s 26ms/step - loss: 1.4308 - accuracy: 0.5077 - val_loss: 1.0149 - val_accuracy: 0.5381
Epoch 2/50
53/53 [==============================] - 0s 6ms/step - loss: 1.0546 - accuracy: 0.5405 - val_loss: 0.8967 - val_accuracy: 0.6262
Epoch 3/50
53/53 [==============================] - 0s 6ms/step - loss: 0.9607 - accuracy: 0.5744 - val_loss: 0.8303 - val_accuracy: 0.6667
Epoch 4/50
53/53 [==============================] - 0s 7ms/step - loss: 0.9245 - accuracy: 0.5970 - val_loss: 0.7756 - val_accuracy: 0.6690
Epoch 5/50
53/53 [==============================] - 0s 7ms/step - loss: 0.8831 - accuracy: 0.6262 - val_loss: 0.7640 - val_accuracy: 0.6333
Epoch 6/50
53/53 [==============================] - 0s 6ms/step - loss: 0.8546 - accuracy: 0.6315 - val_loss: 0.7381 - val_accuracy: 0.6500
Epoch 7/50
53/53 [==============================] - 0s 9ms/step - loss: 0.8547 - accuracy: 0.6357 - val_loss: 0.7286 - val_accuracy: 0.6548
Epoch 8/50
53/53 [=

In [100]:
# Evaluate the model on test data
score = model.evaluate(X_test.values.reshape(X_test.shape[0],1,x_normalised.shape[1],1), y_test, verbose=1)
print('Test accuracy:', score[1])

29/29 [==============================] - 0s 2ms/step - loss: 0.6394 - accuracy: 0.7533
Test accuracy: 0.753333330154419
